In [14]:
import numpy as np
import pandas as pd
from  scipy.optimize import linprog

assets   = ["Stocks","Bonds","Commodities","Cash"]
mu_hat   = np.array([0.038,0.050,0.113,0.001])
Sigma_hat= np.array([[ 0.070,-0.007,0.015, 0.003],
                     [-0.007, 0.033,-0.012, 0.001],
                     [ 0.015,-0.012,0.098,-0.001],
                     [ 0.003, 0.001,-0.001,0.001]])
V0_abs   = 550_000                               # 초기자산($)
V0       = 1.0                                   # 정규화 단위

save0, g, rf = 19_200, 0.03, 0.001               # rf : 현금 기대수익
# t=0…9 불입 → 10년말 가치
S_0_9 = sum(save0*(1+g)**t * (1+rf)**(10-t) for t in range(10))
# t=10 불입(리밸런스 시점)
S10   = save0*(1+g)**10                         
S10_tot = (S_0_9 + S10)/V0_abs                   # 10 년말 추가 자금 (정규화)
# t=11…19 불입 → 20년말 가치 (리스크-프리라 가정; 결정변수에 영향 없음)
S_11_19 = sum(save0*(1+g)**t * (1+rf)**(20-t) for t in range(11,20))
S_fin   = S_11_19/V0_abs                         # 20 년말 상수항

delta = 0.02
states = [(e,b,c) for e in [1,-1] for b in [1,-1] for c in [1,-1]]
mu_scen = np.array([mu_hat + delta*np.array([e,b,c,0]) for (e,b,c) in states])
π0 = np.full(8, 1/8)

np.random.seed(42)
true_id = 0
Rpath = np.random.multivariate_normal(mu_scen[true_id], Sigma_hat, 10)
Σinv   = np.linalg.inv(Sigma_hat)
logL   = np.array([sum(-0.5*(r-μ)@Σinv@(r-μ) for r in Rpath) for μ in mu_scen])
post   = np.exp(logL-logL.max()); post /= post.sum()   

R0 = (1+mu_scen)**10 - 1          # 0-10년 누적
R1 = R0.copy()                    # 10-20년 누적(평균 동일 가정)

nvar = 4 + 8*4                    # x0 + x1^{(s)}
c    = np.zeros(nvar)
for s in range(8):
    for i in range(4):
        c[4+s*4+i] = post[s]*(1+R1[s,i])
c_min = -c                        # linprog는 최소화

Aeq, beq = [], []

# (i) 초기 예산
row = np.zeros(nvar); row[:4] = 1
Aeq.append(row);  beq.append(V0)
for s in range(8):
    row = np.zeros(nvar)
    row[4+s*4:4+s*4+4] = 1
    row[:4] -= (1+R0[s])
    Aeq.append(row); beq.append(S10_tot)

Aeq, beq = np.array(Aeq), np.array(beq)
bounds = [(-10000,10000)]*nvar          # 숏·레버리지 허용

res = linprog(c_min, A_eq=Aeq, b_eq=beq, bounds=bounds, method="highs")
x0_opt = res.x[:4]
x1_opt = res.x[4:].reshape(8,4)
E_V20  = -res.fun + S_fin          # 20년-말 상수 저축분 더하기

print("Optimal x0*  :", dict(zip(assets, x0_opt.round(3))))
print("Expected V_20:", round(E_V20,2), "(per $1 initial)")


Optimal x0*  : {'Stocks': -9999.0, 'Bonds': 10000.0, 'Commodities': 10000.0, 'Cash': -10000.0}
Expected V_20: 53418.59 (per $1 initial)
